<a href="https://colab.research.google.com/github/yukinaga/bayesian_statistics/blob/main/section_5/02_bayes_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  ベイズ推定による分類
PyMC3により、ベイズ推定による分類を実装します。

## pymc3のインストール
pymc3をpipによりインストールします。

In [ ]:
!pip install pymc3

必要な各設定を行います。

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import pymc3 as pm
import seaborn as sns

plt.style.use("seaborn-darkgrid")
plt.rcParams["figure.figsize"] = [12, 9]  # グラフの大きさ

## ロジスティック回帰
今回は、「ロジスティック回帰」による結果を分類します。  
ロジスティック回帰は、以下のように表されます。  

$$\theta = logistic(ax+b)$$

$$y\sim Bernoulli(\theta)$$

ここで、`logistic`は以下の「シグモイド」関数で表されます。

$$logistic(x) = \frac{1}{1+\exp{(-x)}}$$

以下は、シグモイド関数をグラフに描画するコードです。

In [ ]:
x = np.linspace(-12, 12, 100)
logistic = 1 / (1 + np.exp(-x))

plt.plot(x, logistic)
plt.xlabel("$x$")
plt.ylabel("$logistic$")
plt.show()

シグモイド関数は、0と1の間を滑らかに変化します。

## Iris datasetの読み込み
Iris datasetを読み込み、2つの品種のカテゴリ変数を設定します。  

In [ ]:
iris = sns.load_dataset("iris")

df = iris.query("species == ('setosa', 'versicolor')")  # 2品種を使用
y = pd.Categorical(df["species"]).codes  # カテゴリ変数を定義
print("Categories:", y)

x_pl = "petal_length"  # 花びらの長さ
x = df[x_pl].values

plt.scatter(x, y)
plt.xlabel(x_pl)
plt.ylabel(r"$\theta$")
plt.show()

## モデルの構築
PyMC3により、分類のための統計モデルを実装します。  

In [ ]:
with pm.Model() as model:
    a = pm.Normal("a", mu=0, sd=1)
    b = pm.Normal("b", mu=0, sd=1)
  
    mu = pm.math.dot(x, a) + b
    theta = pm.Deterministic("theta", 1 / (1+pm.math.exp(-mu)))  # 後でこの値を使用
    obs = pm.Bernoulli("likehood", p=theta, observed=y)

    trace = pm.sample(4000, chains=2, return_inferencedata=False)

vars_to_see = ["a", "b"]

pm.summary()によりモデルの概要を表示します。

In [ ]:
with model:
    print(pm.summary(trace, vars_to_see, hdi_prob=0.95))

plot_posteriorにより、PyMC3によって推論された事後分布をグラフで表示します。

In [ ]:
with model:
    pm.plot_posterior(trace, vars_to_see, point_estimate='mode')

`pm.plot_trace()`によりサンプリング結果を可視化します。

In [ ]:
with model:
    pm.plot_trace(trace, vars_to_see)

## 分類結果のグラフ表示
グラフを信頼区間と共に描画します。

In [ ]:
theta = trace["theta"].mean(axis=0)
idx = np.argsort(x)

theta_hdi = pm.hdi(trace["theta"], hdi_prob=0.95)[idx]
plt.fill_between(x[idx], theta_hdi[:,0], theta_hdi[:,1], color="r", alpha=0.4)

plt.plot(x[idx], theta[idx], color="r")

plt.scatter(x, y)

plt.xlabel(x_pl)
plt.ylabel(r"$\theta$")
plt.show() 